*Los modelos predictivos en el ámbito del machine learning representan una herramienta invaluable para el análisis y la mejora continua de las operaciones empresariales. En particular, en este proyecto centrado en el sector de las tarjetas de crédito, se explorará la aplicación de técnicas predictivas para evaluar la probabilidad de abandono de clientes del banco. A través de la identificación de patrones significativos en los datos, se buscará comprender mejor el comportamiento de los clientes y anticipar posibles acciones de abandono. Posteriormente, se implementarán medidas estratégicas con el fin de mitigar estos riesgos y fomentar la retención de clientes. Este enfoque no solo busca optimizar la rentabilidad del negocio, sino también fortalecer la relación con los clientes al ofrecer soluciones personalizadas y proactivas.*

*Medida elegida: *La precisión* en problemas de modelos supervisados  -  clasificación indica qué tan exacto es un modelo en predecir la clase correcta. Por ejemplo, en la detección de fraudes con tarjetas de crédito, una alta precisión significa que el modelo identifica correctamente la mayoría de las transacciones fraudulentas, evitando así errores costosos. En resumen, optimizar la precisión nos ayuda a mejorar la confiabilidad de nuestras predicciones, lo que es crucial para la seguridad financiera y la satisfacción del cliente.*

*Imagina que estás trabajando en un proyecto para predecir si un cliente abandonará un banco. La precisión en este caso sería la medida de cuántos de los clientes que el modelo predice que van a abandonar realmente lo hacen. Por ejemplo, si el modelo predice que 100 clientes abandonarán el banco y 85 de ellos realmente lo hacen, la precisión sería del 85%. Una alta precisión aquí significa que el banco puede identificar con confianza a la mayoría de los clientes propensos a abandonarlo, permitiéndoles tomar medidas proactivas para retenerlos y optimizar recursos.*

<div style="text-align:center;">
    <img src="img\precision.jpg" alt="Texto alternativo" width=500px >
</div>

### **Librerías**

In [1]:
### Manipulación de Datos
import pandas as pd
import numpy as np

### Visualización de Datos
import seaborn as sns
import matplotlib.pyplot as plt

### Tratamiento de datos
from utils.funciones import extended_describe

### Machine Learning
# Preparación de datos 
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from imblearn.over_sampling import SMOTE
from sklearn.decomposition import PCA

# Modelos
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import  RandomForestClassifier,  GradientBoostingClassifier, VotingClassifier,  AdaBoostClassifier, StackingClassifier, HistGradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from catboost import CatBoostClassifier
import lightgbm as lgb
import xgboost as xgb
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import  classification_report, confusion_matrix,  accuracy_score, f1_score,  precision_score, recall_score, \
roc_curve, roc_auc_score, ConfusionMatrixDisplay, multilabel_confusion_matrix, make_scorer


from sklearn.model_selection import GridSearchCV
import pickle
from utils.funciones import BaseLine

# Ignorar warnings
import warnings
warnings.filterwarnings('ignore')
df = pd.read_csv('./data/processed/Churn_Modelling.csv')

### **Modelo 1**
**Sin feature enginering y con OneHotEncoder**


Antes de iniciar cualquier análisis de datos, es crucial comprender la estructura y la naturaleza de los datos en su forma original. Esto proporciona una base sólida para evitar errores comunes, como el sobreajuste y la mala interpretación de los resultados. Es esencial establecer una referencia clara de cómo se reciben los datos inicialmente, ya que cualquier manipulación posterior podría influir en los resultados finales.

In [3]:
df = pd.read_csv('./data/processed/Churn_Modelling.csv')
df.drop(columns=['RowNumber', 'CustomerId', 'Surname',], inplace=True)

# Definir las variables X e y
X = df.drop(columns=['Exited'])
y = df['Exited']

numeric_features = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']
variables_sin_modificacion = ['HasCrCard', 'IsActiveMember']
categorical_features = ['Geography', 'Gender']

# Definir transformadores para características numéricas y categóricas
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder()

# Crear un ColumnTransformer para aplicar transformaciones a diferentes columnas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Crear el pipeline con el preprocesador
pipeline = make_pipeline(preprocessor)
# Aplicar el preprocesador a los datos
X_processed = pipeline.fit_transform(X)

# Obtener los nombres de las columnas después de aplicar OneHotEncoder
encoded_categorical_columns = preprocessor.named_transformers_['cat']\
    .get_feature_names_out(input_features=categorical_features)

# Combinar los nombres de las columnas numéricas y categóricas
processed_columns = numeric_features + list(encoded_categorical_columns)

# Crear DataFrame con los datos procesados y los nombres de las columnas
processed_df = pd.DataFrame(X_processed, columns=processed_columns)
concatenated_series  = pd.concat((processed_df, X[[ 'HasCrCard', 'IsActiveMember']]), axis=1)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(concatenated_series, y, test_size=0.2, random_state=24)

# Aplicar SMOTE solo al conjunto de entrenamiento
smote = SMOTE(random_state=24)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

BaseLine(X_train_resampled, y_train_resampled, cv=5, metricas_cross_validate= ['accuracy', 'f1_macro', 'recall_macro', 'precision_macro']) # Realizar un baseline -> input = todos


Learning rate set to 0.027773
0:	learn: 0.6759898	total: 214ms	remaining: 3m 34s
1:	learn: 0.6593409	total: 255ms	remaining: 2m 7s
2:	learn: 0.6454645	total: 291ms	remaining: 1m 36s
3:	learn: 0.6311764	total: 346ms	remaining: 1m 26s
4:	learn: 0.6209109	total: 389ms	remaining: 1m 17s
5:	learn: 0.6094650	total: 444ms	remaining: 1m 13s
6:	learn: 0.5977251	total: 498ms	remaining: 1m 10s
7:	learn: 0.5878024	total: 541ms	remaining: 1m 7s
8:	learn: 0.5773729	total: 583ms	remaining: 1m 4s
9:	learn: 0.5702681	total: 630ms	remaining: 1m 2s
10:	learn: 0.5607843	total: 673ms	remaining: 1m
11:	learn: 0.5520154	total: 708ms	remaining: 58.3s
12:	learn: 0.5449172	total: 741ms	remaining: 56.3s
13:	learn: 0.5390415	total: 774ms	remaining: 54.5s
14:	learn: 0.5335363	total: 807ms	remaining: 53s
15:	learn: 0.5269017	total: 844ms	remaining: 51.9s
16:	learn: 0.5223960	total: 884ms	remaining: 51.1s
17:	learn: 0.5161423	total: 921ms	remaining: 50.3s
18:	learn: 0.5102834	total: 965ms	remaining: 49.9s
19:	learn:

,Modelo,Score
34,XGBoost_recall,0.885894
35,XGBoost_precision,0.902164
33,XGBoost_f1,0.882241
32,XGBoost_accuracy,0.885912
42,SVC_recall,0.817594
43,SVC_precision,0.818084
41,SVC_f1,0.817520
40,SVC_accuracy,0.817597
6,Random Forest_recall,0.899936
7,Random Forest_precision,0.902839


La mayoría de nuestros datos se ajustan adecuadamente según la media obtenida mediante la validación cruzada. Esta técnica implica dividir el conjunto de datos en múltiples subconjuntos, alternando entre ellos para entrenar y evaluar el modelo. La media de las métricas de rendimiento proporciona una evaluación más robusta del modelo, ya que no depende de una sola división de los datos. Es importante recordar que los valores en el conjunto de datos son aproximados y redondeados, lo que facilita la aplicación de modelos y la selección de los más adecuados.

La validación cruzada es esencial para evaluar la capacidad de generalización de un modelo. Consiste en dividir el conjunto de datos en k subconjuntos (llamados "folds"), donde uno de los subconjuntos se reserva como conjunto de prueba y los restantes se utilizan como conjunto de entrenamiento. Este proceso se repite k veces, de manera que cada subconjunto se utilice como conjunto de prueba exactamente una vez. Finalmente, se promedian los resultados de evaluación para obtener una estimación más precisa del rendimiento del modelo en datos no vistos. Esto ayuda a evitar el sobreajuste al evaluar el modelo en diferentes conjuntos de datos y garantiza una evaluación más robusta de su capacidad de generalización.

<div style="text-align:center;">
    <img src="img\cross_validate_explicacion.png" alt="Texto alternativo" width=500px >
</div>


|Métricas | Valores |
|------|-------|
|Random Forest_accuracy|	0.905824|
|Random Forest_f1_macro|	0.905501|
|Random Forest_recall_macro|	0.905816|
|Random Forest_precision_macro|	0.909381|

<br>

|Métricas | Valores |
|--------|--------|
|XGBoost_recall|	0.885894|
|XGBoost_precision|	0.902164|
|XGBoost_f1| 0.882241|
|XGBoost_accuracy|	0.885912|

<br>

|Métricas | Valores |
|-----|------------|
|CatBoost_recall|	0.889580|
|CatBoost_precision|	0.909304|
|CatBoost_f1|	0.884705|
|CatBoost_accuracy|	0.889599|

<br>

|Métricas | Valores |
|----|-------------|
|LGBM_recall|	0.888168|
|LGBM_precision|	0.906428|
|LGBM_f1|	0.883723|
|LGBM_accuracy|	0.888187|

In [6]:
# grid_1 = {
#     'n_estimators': [50, 100, 200],
#     'max_depth': [20, 30, None],
#     'min_samples_split': [2, 10],
#     'min_samples_leaf': [1, 5, 10],
#     'max_features': ['auto', 'sqrt'],
#     'bootstrap': [True, False]
# }

# forest = RandomForestClassifier()

# grd_search_forest = GridSearchCV(forest,
#                            grid_1,
#                            cv=5,
#                            scoring='precision_macro',
#                            n_jobs=-1
#                           )

# grd_search_forest.fit(X_train_resampled, y_train_resampled)



GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'max_depth': [20, 30, None],
                         'max_features': ['auto', 'sqrt'],
                         'min_samples_leaf': [1, 5, 10],
                         'min_samples_split': [2, 10],
                         'n_estimators': [50, 100, 200]},
             scoring='precision_macro')

In [57]:
# print('Los mejores hiperparámetros obtenidos:', grd_search_forest.best_params_)
# print(classification_report(y_test, grd_search_forest.predict(X_test)))

# # Guardar el modelo en un archivo
# with open('modelo_forest_1.pkl', 'wb') as f:
#     pickle.dump(grd_search_forest, f)

# # Cargar el modelo desde el archivo
with open('./modelos/modelo_forest_1.pkl', 'rb') as f:
    loaded_model = pickle.load(f)
    
# # Utilizar el modelo cargado para hacer predicciones
# predictions = loaded_model.predict(X_test)

# # Ejemplo de cómo utilizar el modelo cargado
# print("Predicciones realizadas con el modelo cargado:")
# print(predictions)

In [58]:
print(classification_report(y_test, loaded_model.predict(X_test)))

              precision    recall  f1-score   support

           0       0.89      0.93      0.91      1587
           1       0.66      0.55      0.60       413

    accuracy                           0.85      2000
   macro avg       0.78      0.74      0.75      2000
weighted avg       0.84      0.85      0.84      2000



 Los mejores hiperparámetros del randomForest fueron : *{'bootstrap': **False**, 'max_depth': **30**, 'max_features': **'sqrt'**, 'min_samples_leaf': **1**, 'min_samples_split': **2**, 'n_estimators': **200**}*
 
Los resultados de la validación cruzada revelaron una disminución en las métricas, lo cual podría atribuirse al sobreajuste inherente del algoritmo de Random Forest si no se realiza un adecuado proceso de poda. Sin embargo, a pesar de este desafío, nuestras métricas actuales no son desalentadoras. Es evidente que existe margen para mejorar especialmente la precisión en las características del uno, que actualmente se sitúa en un 66%. Nuestro objetivo es alcanzar un nivel de precisión del 90% o incluso del 94%, lo que nos impulsará hacia una mayor confiabilidad en las predicciones y una toma de decisiones más efectiva. En resumen, el enfoque se centrará en implementar estrategias de poda adecuadas para mitigar el sobreajuste y en afinar aún más el modelo para alcanzar nuestras metas de precisión.

Hemos adoptado una estrategia de almacenamiento de modelos para evitar el sobreprocesamiento. Así optimizamos nuestros recursos y garantizamos una selección eficiente del modelo más adecuado.

In [9]:
# grid_2 = {
#     'n_estimators': [100, 150, 200],
#     'learning_rate': [0.01, 0.05],
#     'max_depth': [4, 6, 8],
#     'subsample': [0.7, 0.9],
#     'colsample_bytree': [0.7, 0.9]
# }


# xgbBoost = xgb.XGBClassifier()

# grd_search_xgbBoost = GridSearchCV(xgbBoost,
#                            grid_2,
#                            cv=5,
#                            scoring='precision_macro',
#                            n_jobs=-1
#                           )

# grd_search_xgbBoost.fit(X_train_resampled, y_train_resampled)


GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.7, 0.9],
                         'learning_rate': [0.01, 0.05], 'max_depth': [4, 6, 8],
                         'n_estimators': [100, 150, 200],
                         'subsample': [0.7, 0.9]},
             scoring='precision_macro')

In [59]:
# # Guardar el modelo en un archivo
# with open('./modelos/modelo_xgbCla_1.pkl', 'wb') as f:
#     pickle.dump(grd_search_xgbBoost, f)
    
# # # Cargar el modelo desde el archivo
with open('./modelos/modelo_xgbCla_1.pkl', 'rb') as f:
    loaded_model_xgb = pickle.load(f)


In [60]:
print(classification_report(y_test, loaded_model_xgb.predict(X_test)))

              precision    recall  f1-score   support

           0       0.89      0.93      0.91      1587
           1       0.69      0.57      0.62       413

    accuracy                           0.86      2000
   macro avg       0.79      0.75      0.77      2000
weighted avg       0.85      0.86      0.85      2000



Los mejores hiperparámetros del XGBClassifier fueron : *{'colsample_bytree': **0.7**, 'learning_rate': **0.05**, 'max_depth': **8**, 'n_estimators':**200**, 'subsample': **0.7**}*

Las métricas mejoraron al emplear el XGBoost Classifier en comparación con el Random Forest. Esta mejora indica la eficacia del XGBoost en nuestro conjunto de datos y resalta su capacidad para generar modelos más precisos.

### **Modelo 2**
**Con feature enginering y OneHotEncoder**

In [61]:
df = pd.read_csv('./data/processed/Churn_Modelling.csv')
# Este ratio te daría una medida de la proporción del saldo en la cuenta en relación con el salario estimado anual del cliente. 
# Calcular el ratio entre el saldo en la cuenta y el salario estimado anual
df['Saldo_Salario_Ratio'] = df['Balance'] / df['EstimatedSalary']
# Agregar una pequeña cantidad a la antigüedad para evitar la división por cero
df['Balance_Tenure_Ratio'] = df['Balance'] / (df['Tenure'] + 1e-6)

# Definir las variables X e y
X = df.drop(columns=['RowNumber', 'CustomerId', 'Surname', 'Exited', 'Tenure', 'Balance']) # Evitamos la colinealidad
y = df['Exited']

numeric_features = ['CreditScore', 'Age', 'NumOfProducts',  'EstimatedSalary', 'Saldo_Salario_Ratio', 'Balance_Tenure_Ratio']
categorical_features = ['Geography', 'Gender']

# Definir transformadores para características numéricas y categóricas
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder()

# Crear un ColumnTransformer para aplicar transformaciones a diferentes columnas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Crear el pipeline con el preprocesador
pipeline = make_pipeline(preprocessor)

X_processed = pipeline.fit_transform(X)

# Obtener los nombres de las columnas después de aplicar OneHotEncoder
encoded_categorical_columns = preprocessor.named_transformers_['cat']\
    .get_feature_names_out(input_features=categorical_features)

# Combinar los nombres de las columnas numéricas y categóricas
processed_columns = numeric_features + list(encoded_categorical_columns)

# Crear DataFrame con los datos procesados y los nombres de las columnas
processed_df = pd.DataFrame(X_processed, columns=processed_columns)
concatenated_series  = pd.concat((processed_df, X[[ 'HasCrCard', 'IsActiveMember']]), axis=1)

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(concatenated_series, y, test_size=0.2, random_state=24)

# Aplicar SMOTE solo al conjunto de entrenamiento
smote = SMOTE(random_state=24)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

BaseLine(X_train_resampled, y_train_resampled, cv=5, metricas_cross_validate= ['accuracy', 'f1_macro', 'recall_macro', 'precision_macro']) # Realizar BaseLine -> input = todos

Learning rate set to 0.027773
0:	learn: 0.6751289	total: 38.5ms	remaining: 38.5s
1:	learn: 0.6587682	total: 71.6ms	remaining: 35.7s
2:	learn: 0.6434785	total: 97.6ms	remaining: 32.4s
3:	learn: 0.6296203	total: 125ms	remaining: 31s
4:	learn: 0.6185872	total: 154ms	remaining: 30.6s
5:	learn: 0.6063067	total: 182ms	remaining: 30.2s
6:	learn: 0.5951124	total: 209ms	remaining: 29.7s
7:	learn: 0.5839602	total: 239ms	remaining: 29.6s
8:	learn: 0.5741502	total: 267ms	remaining: 29.4s
9:	learn: 0.5649041	total: 298ms	remaining: 29.5s
10:	learn: 0.5577002	total: 371ms	remaining: 33.4s
11:	learn: 0.5492929	total: 455ms	remaining: 37.5s
12:	learn: 0.5431407	total: 497ms	remaining: 37.7s
13:	learn: 0.5377614	total: 541ms	remaining: 38.1s
14:	learn: 0.5317217	total: 598ms	remaining: 39.3s
15:	learn: 0.5257178	total: 641ms	remaining: 39.4s
16:	learn: 0.5205635	total: 683ms	remaining: 39.5s
17:	learn: 0.5157550	total: 717ms	remaining: 39.1s
18:	learn: 0.5114186	total: 752ms	remaining: 38.8s
19:	learn:

,Modelo,Score
34,XGBoost_recall,0.868093
35,XGBoost_precision,0.876113
33,XGBoost_f1,0.866505
32,XGBoost_accuracy,0.868107
42,SVC_recall,0.792344
43,SVC_precision,0.793586
41,SVC_f1,0.792119
40,SVC_accuracy,0.792346
6,Random Forest_recall,0.883860
7,Random Forest_precision,0.885967


Hemos observado que la inclusión de nuevas características no produjo los resultados esperados en la mejora de nuestros modelos. Esta constatación resalta la importancia de la selección cuidadosa de características en el proceso de modelado. En lugar de simplemente agregar más variables, debemos enfocarnos en identificar aquellas que aporten un valor significativo y contribuyan de manera efectiva a la precisión del modelo. 

<br>

|Métricas | Valores |
|--------|--------|
|Random Forest_recall|	0.885585|
|Random Forest_precision|	0.887898|
|Random Forest_f1|	0.885408|
|Random Forest_accuracy|	0.885590|

<br>

|Métricas | Valores |
|--------|--------|
|LGBM_recall|	0.874132|
|LGBM_precision|	0.886193|
|LGBM_f1|	0.871222|
|LGBM_accuracy|	0.874148|

<br>

|Métricas | Valores |
|--------|--------|
|CatBoost_recall|	0.880327|
|CatBoost_precision|	0.894255|
|CatBoost_f1|	0.876986|
|CatBoost_accuracy|	0.880344|

<br>

|Métricas | Valores |
|--------|--------|
|XGBoost_recall|	0.868093|
|XGBoost_precision|	0.876113|
|XGBoost_f1|	0.866505|
|XGBoost_accuracy|	0.868107|

In [18]:
# grid_3 = {
#     'n_estimators': [50, 100, 200],
#     'max_depth': [10, 20, None],
#     'min_samples_split': [2, 10],
#     'max_features': ['auto', 'sqrt'],
#     'bootstrap': [True, False]
# }

# forest_conFeature = RandomForestClassifier()

# grd_search_forest_conFeature = GridSearchCV(forest_conFeature,
#                            grid_3,
#                            cv=5,
#                            scoring='precision_macro',
#                            n_jobs=-1
#                           )

# grd_search_forest_conFeature.fit(X_train_resampled, y_train_resampled)

GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'max_depth': [10, 20, None],
                         'max_features': ['auto', 'sqrt'],
                         'min_samples_split': [2, 10],
                         'n_estimators': [50, 100, 200]},
             scoring='precision_macro')

In [62]:
# grd_search_forest_conFeature.best_params_
# print(classification_report(y_test, grd_search_forest_conFeature.predict(X_test)))

# # Guardar el modelo en un archivo
# with open('./modelos/modelo_forest_Confeature_1.pkl', 'wb') as f:
#     pickle.dump(grd_search_forest_conFeature, f)
    
# # # Cargar el modelo desde el archivo
with open('./modelos/modelo_forest_Confeature_1.pkl', 'rb') as f:
    loaded_model_xgb = pickle.load(f)

In [63]:
print(classification_report(y_test, loaded_model_xgb.predict(X_test)))

              precision    recall  f1-score   support

           0       0.89      0.90      0.89      1587
           1       0.59      0.58      0.58       413

    accuracy                           0.83      2000
   macro avg       0.74      0.74      0.74      2000
weighted avg       0.83      0.83      0.83      2000



Los mejores hiperparametros son : *{'bootstrap': False, 'max_depth': None, 'max_features': 'sqrt', 'min_samples_split': 2, 'n_estimators': 200}*



*************

### Modelo 3 - CatBoost
**Con feature enginering**

El CatBoost es un conjunto de árboles de decisión que destaca por su capacidad para manejar variables categóricas sin necesidad de preprocesamiento adicional. A diferencia de la mayoría de los modelos, que requieren la conversión de variables categóricas a numéricas, el CatBoost puede trabajar directamente con ellas, lo que simplifica significativamente el proceso de modelado. En el caso de nuestro baseline, hemos observado que el CatBoost proporciona resultados satisfactorios incluso cuando se utiliza con variables numéricas, lo que nos motiva a explorar su desempeño sin necesidad de procesar las características previamente. Esta decisión nos permitirá aprovechar al máximo las capacidades inherentes del CatBoost y potencialmente mejorar la eficacia de nuestro modelo.

In [64]:
df = pd.read_csv('./data/processed/Churn_Modelling.csv')
# Este ratio te daría una medida de la proporción del saldo en la cuenta en relación con el salario estimado anual del cliente. 
# Calcular el ratio entre el saldo en la cuenta y el salario estimado anual
df['Saldo_Salario_Ratio'] = df['Balance'] / df['EstimatedSalary']
# Agregar una pequeña cantidad a la antigüedad para evitar la división por cero
df['Balance_Tenure_Ratio'] = df['Balance'] / (df['Tenure'] + 1e-6)

# Definir las variables X e y
X = df.drop(columns=['RowNumber', 'CustomerId', 'Surname', 'Exited', 'Tenure', 'Balance']) # Evitamos la colinealidad
y = df['Exited']

numeric_features = ['CreditScore', 'Age', 'NumOfProducts',  'EstimatedSalary', 'Saldo_Salario_Ratio', 'Balance_Tenure_Ratio']
categorical_features = ['Geography', 'Gender']

# Definir transformadores para características numéricas y categóricas
numeric_transformer = StandardScaler()

# Crear un ColumnTransformer para aplicar transformaciones a diferentes columnas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)
    ]
)

# Crear el pipeline con el preprocesador
pipeline = make_pipeline(preprocessor)

X_processed = pipeline.fit_transform(X)

# Crear DataFrame con los datos procesados y los nombres de las columnas
processed_df = pd.DataFrame(X_processed, columns=numeric_features)
concatenated_series  = pd.concat((processed_df, X[[ 'HasCrCard', 'IsActiveMember', 'Geography', 'Gender']]), axis=1)
concatenated_series

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(concatenated_series, y, test_size=0.2, random_state=24)

# # Aplicar SMOTE solo al conjunto de entrenamiento
# smote = SMOTE(random_state=24)
# X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train) # el smote no acepta categoricos por ende el catboost esperemos que den buenos resultados con las clases desbalanceadas

# MODELO
# model_4 = CatBoostClassifier()
# model_4.fit(X_train, y_train, cat_features=categorical_features) # cat_features es una lista de índices de columnas categóricas para que CatBoost las procese adecuadamente. 


In [65]:
# # Guardar el modelo en un archivo
# with open('./modelos/modelo_CatBoost_Confeature_1.pkl', 'wb') as f:
#     pickle.dump(model_4, f)
    
# # # Cargar el modelo desde el archivo
with open('./modelos/modelo_CatBoost_Confeature_1.pkl', 'rb') as f:
    loaded_model_xgb = pickle.load(f)


# Hacer predicciones
print(classification_report(y_test, loaded_model_xgb.predict(X_test))) 

              precision    recall  f1-score   support

           0       0.88      0.96      0.92      1587
           1       0.76      0.47      0.58       413

    accuracy                           0.86      2000
   macro avg       0.82      0.72      0.75      2000
weighted avg       0.85      0.86      0.85      2000



El CatBoost muestra resultados consistentes o, en muchos casos, superiores a otros algoritmos en términos de estabilidad y rendimiento, especialmente cuando se trata de datos con variables categóricas. Esto sugiere que CatBoost puede ser una opción sólida para nuestros modelos, ya que proporciona una base confiable y eficaz para nuestras predicciones.

### **Modelo 4** - CatBoost
**Sin features**

In [67]:
df = pd.read_csv('./data/processed/Churn_Modelling.csv')

# Definir las variables X e y
X = df.drop(columns=['RowNumber', 'CustomerId', 'Surname', 'Exited']) # Evitamos la colinealidad
y = df['Exited']

numeric_features = ['CreditScore', 'Age', 'NumOfProducts', 'Tenure', 'Balance', 'EstimatedSalary']
categorical_features = ['Geography', 'Gender']

# Definir transformadores para características numéricas y categóricas
numeric_transformer = StandardScaler()

# Crear un ColumnTransformer para aplicar transformaciones a diferentes columnas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)
    ]
)

# Crear el pipeline con el preprocesador
pipeline = make_pipeline(preprocessor)

X_processed = pipeline.fit_transform(X)

# Crear DataFrame con los datos procesados y los nombres de las columnas
processed_df = pd.DataFrame(X_processed, columns=numeric_features)
concatenated_series  = pd.concat((processed_df, X[[ 'HasCrCard', 'IsActiveMember', 'Geography', 'Gender']]), axis=1)
concatenated_series

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(concatenated_series, y, test_size=0.2, random_state=24)

# MODELO
model_5 = CatBoostClassifier()
model_5.fit(X_train, y_train, cat_features=categorical_features)

Learning rate set to 0.025035
0:	learn: 0.6710085	total: 110ms	remaining: 1m 50s
1:	learn: 0.6507395	total: 230ms	remaining: 1m 54s
2:	learn: 0.6323542	total: 325ms	remaining: 1m 47s


3:	learn: 0.6161050	total: 455ms	remaining: 1m 53s
4:	learn: 0.6013040	total: 606ms	remaining: 2m
5:	learn: 0.5853518	total: 777ms	remaining: 2m 8s
6:	learn: 0.5717602	total: 902ms	remaining: 2m 7s
7:	learn: 0.5577596	total: 996ms	remaining: 2m 3s
8:	learn: 0.5464651	total: 1.09s	remaining: 2m
9:	learn: 0.5336771	total: 1.17s	remaining: 1m 56s
10:	learn: 0.5228673	total: 1.27s	remaining: 1m 53s
11:	learn: 0.5124554	total: 1.36s	remaining: 1m 52s
12:	learn: 0.5019303	total: 1.44s	remaining: 1m 49s
13:	learn: 0.4945155	total: 1.54s	remaining: 1m 48s
14:	learn: 0.4850131	total: 1.63s	remaining: 1m 46s
15:	learn: 0.4773584	total: 1.7s	remaining: 1m 44s
16:	learn: 0.4694568	total: 1.81s	remaining: 1m 44s
17:	learn: 0.4645598	total: 1.88s	remaining: 1m 42s
18:	learn: 0.4581376	total: 1.96s	remaining: 1m 41s
19:	learn: 0.4526324	total: 2.04s	remaining: 1m 40s
20:	learn: 0.4474730	total: 2.14s	remaining: 1m 39s
21:	learn: 0.4428551	total: 2.23s	remaining: 1m 39s
22:	learn: 0.4371247	total: 2.3

In [72]:
# # Guardar el modelo en un archivo
# with open('./modelos/modelo_CatBoost_Sinfeature_2.pkl', 'wb') as f:
#     pickle.dump(model_5, f)
    
# # # Cargar el modelo desde el archivo
with open('./modelos/modelo_CatBoost_Sinfeature_2.pkl', 'rb') as f:
    loaded_model_xgb = pickle.load(f)

# Hacer predicciones
print(classification_report(y_test, loaded_model_xgb.predict(X_test))) 

              precision    recall  f1-score   support

           0       0.88      0.97      0.92      1587
           1       0.79      0.48      0.59       413

    accuracy                           0.87      2000
   macro avg       0.83      0.72      0.76      2000
weighted avg       0.86      0.87      0.85      2000



### **Modelo 5**
**Aplicaremos el modelo PCA para la reducción de complejidad de nuestro modelo.** 

La reducción de dimensionalidad se utiliza para simplificar conjuntos de datos con muchas características, lo que ayuda a abordar problemas como el sobreajuste y la complejidad computacional. El PCA (Análisis de Componentes Principales) es una técnica común de reducción de dimensionalidad que identifica las combinaciones lineales de características que capturan la mayor parte de la variabilidad en los datos. Esto permite conservar la información importante mientras se reduce el número de características, lo que facilita la visualización, la interpretación y el procesamiento de los datos, especialmente en conjuntos de datos de alta dimensión.

<div style="text-align:center;">
    <img src="img\pca.jpg" alt="Texto alternativo" width=500px >
</div>

In [73]:
df = pd.read_csv('./data/processed/Churn_Modelling.csv')
df.drop(columns=['RowNumber', 'CustomerId', 'Surname'], inplace=True)

# Definimos variables X -> features , y -> Target
X = df.drop(columns='Exited')
y = df['Exited']

numeric_features = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']
categorical_features = ['Geography', 'Gender']


# Definir transformadores para características numéricas y categóricas
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder()

# Crear un ColumnTransformer para aplicar transformaciones a diferentes columnas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)
    ]
)

# Crear el pipeline con el preprocesador
pipeline = make_pipeline(preprocessor)
# Aplicar el preprocesador a los datos
X_processed = pipeline.fit_transform(X)

# Crear DataFrame con los datos procesados y los nombres de las columnas
processed_df = pd.DataFrame(X_processed, columns=numeric_features)
concatenated_series  = pd.concat((processed_df, X[[ 'HasCrCard', 'IsActiveMember']]), axis=1)

pca = PCA(n_components=7)
X10D = pca.fit_transform(concatenated_series)
print(pca.explained_variance_ratio_)

[0.20303961 0.1571695  0.15505024 0.15372083 0.15318949 0.10748379
 0.03823181]


In [75]:
df_catboost = pd.concat((pd.DataFrame(X10D), X[['Geography', 'Gender']]), axis=1)

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(df_catboost, y, test_size=0.2, random_state=24)

# Crear y entrenar el modelo CatBoost con pesos de clase
model_6 = CatBoostClassifier(iterations=100, learning_rate=0.1, cat_features=categorical_features)

model_6.fit(X_train, y_train)

0:	learn: 0.6483011	total: 145ms	remaining: 14.3s
1:	learn: 0.6104936	total: 242ms	remaining: 11.9s
2:	learn: 0.5792335	total: 336ms	remaining: 10.9s
3:	learn: 0.5523911	total: 532ms	remaining: 12.8s
4:	learn: 0.5279452	total: 716ms	remaining: 13.6s
5:	learn: 0.5086243	total: 919ms	remaining: 14.4s
6:	learn: 0.4943534	total: 1.11s	remaining: 14.8s
7:	learn: 0.4785671	total: 1.24s	remaining: 14.3s
8:	learn: 0.4663019	total: 1.41s	remaining: 14.2s


9:	learn: 0.4535767	total: 1.57s	remaining: 14.1s
10:	learn: 0.4433746	total: 1.77s	remaining: 14.3s
11:	learn: 0.4348608	total: 1.92s	remaining: 14.1s
12:	learn: 0.4266651	total: 2.13s	remaining: 14.3s
13:	learn: 0.4191817	total: 2.43s	remaining: 14.9s
14:	learn: 0.4138053	total: 2.78s	remaining: 15.7s
15:	learn: 0.4078733	total: 2.89s	remaining: 15.2s
16:	learn: 0.4022968	total: 3.13s	remaining: 15.3s
17:	learn: 0.3983066	total: 3.35s	remaining: 15.3s
18:	learn: 0.3941745	total: 3.55s	remaining: 15.1s
19:	learn: 0.3898004	total: 3.92s	remaining: 15.7s
20:	learn: 0.3861519	total: 4.15s	remaining: 15.6s
21:	learn: 0.3832285	total: 4.33s	remaining: 15.4s
22:	learn: 0.3799949	total: 4.52s	remaining: 15.1s
23:	learn: 0.3771273	total: 4.67s	remaining: 14.8s
24:	learn: 0.3735400	total: 4.77s	remaining: 14.3s
25:	learn: 0.3691745	total: 4.87s	remaining: 13.9s
26:	learn: 0.3653659	total: 5.03s	remaining: 13.6s
27:	learn: 0.3616868	total: 5.16s	remaining: 13.3s
28:	learn: 0.3583510	total: 5.3s

In [77]:
# # Guardar el modelo en un archivo
# with open('./modelos/modelo_CatBoost_sinfeature_1.pkl', 'wb') as f:
#     pickle.dump(model_6, f)
    
# # # Cargar el modelo desde el archivo
with open('./modelos/modelo_CatBoost_sinfeature_1.pkl', 'rb') as f:
    loaded_model_xgb = pickle.load(f)
    

print(classification_report(y_test, loaded_model_xgb.predict(X_test)))

              precision    recall  f1-score   support

           0       0.86      0.97      0.91      1587
           1       0.78      0.40      0.53       413

    accuracy                           0.85      2000
   macro avg       0.82      0.68      0.72      2000
weighted avg       0.84      0.85      0.83      2000



In [93]:
df = pd.read_csv('./data/processed/Churn_Modelling.csv')
df.drop(columns=['RowNumber', 'CustomerId', 'Surname'], inplace=True)

# Definimos variables X -> features , y -> Target
X = df.drop(columns='Exited')
y = df['Exited']

numeric_features = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']
categorical_features = ['Geography', 'Gender']


# Definir transformadores para características numéricas y categóricas
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder()

# Crear un ColumnTransformer para aplicar transformaciones a diferentes columnas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)
    ]
)

# Crear el pipeline con el preprocesador
pipeline = make_pipeline(preprocessor)
# Aplicar el preprocesador a los datos
X_processed = pipeline.fit_transform(X)

# Crear DataFrame con los datos procesados y los nombres de las columnas
processed_df = pd.DataFrame(X_processed, columns=numeric_features)
concatenated_series  = pd.concat((processed_df, X[[ 'HasCrCard', 'IsActiveMember','Geography', 'Gender']]), axis=1)

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(concatenated_series, y, test_size=0.2, random_state=24)

# Crear y entrenar el modelo CatBoost con pesos de clase
model_7 = CatBoostClassifier(class_weight='balanced'. iterations=200, learning_rate=0.05, cat_features=categorical_features)

model_7.fit(X_train, y_train)

0:	learn: 0.6863094	total: 88.8ms	remaining: 17.7s
1:	learn: 0.6797070	total: 171ms	remaining: 16.9s
2:	learn: 0.6730847	total: 277ms	remaining: 18.2s
3:	learn: 0.6669042	total: 367ms	remaining: 18s
4:	learn: 0.6612002	total: 525ms	remaining: 20.5s


5:	learn: 0.6547692	total: 622ms	remaining: 20.1s
6:	learn: 0.6502872	total: 689ms	remaining: 19s
7:	learn: 0.6443539	total: 791ms	remaining: 19s
8:	learn: 0.6384062	total: 1.02s	remaining: 21.6s
9:	learn: 0.6329896	total: 1.23s	remaining: 23.5s
10:	learn: 0.6277055	total: 1.52s	remaining: 26.2s
11:	learn: 0.6223455	total: 1.74s	remaining: 27.2s
12:	learn: 0.6170796	total: 2s	remaining: 28.8s
13:	learn: 0.6118903	total: 2.16s	remaining: 28.8s
14:	learn: 0.6064165	total: 2.31s	remaining: 28.5s
15:	learn: 0.6010568	total: 2.43s	remaining: 27.9s
16:	learn: 0.5960022	total: 2.53s	remaining: 27.3s
17:	learn: 0.5912386	total: 2.62s	remaining: 26.5s
18:	learn: 0.5865547	total: 2.71s	remaining: 25.8s
19:	learn: 0.5819772	total: 2.8s	remaining: 25.2s
20:	learn: 0.5779696	total: 2.9s	remaining: 24.7s
21:	learn: 0.5735675	total: 2.99s	remaining: 24.2s
22:	learn: 0.5690743	total: 3.1s	remaining: 23.8s
23:	learn: 0.5654835	total: 3.18s	remaining: 23.3s
24:	learn: 0.5614796	total: 3.26s	remaining: 2

In [94]:
print(classification_report(y_test, model_7.predict(X_test)))

              precision    recall  f1-score   support

           0       0.86      0.98      0.92      1587
           1       0.84      0.40      0.54       413

    accuracy                           0.86      2000
   macro avg       0.85      0.69      0.73      2000
weighted avg       0.86      0.86      0.84      2000



In [79]:
# # Guardar el modelo en un archivo
# with open('./modelos/modelo_CatBoost_Confeature_1.pkl', 'wb') as f:
#     pickle.dump(model_7, f)
    
# # # Cargar el modelo desde el archivo
with open('./modelos/modelo_CatBoost_Confeature_1.pkl', 'rb') as f:
    loaded_model_xgb = pickle.load(f)


print(classification_report(y_test, loaded_model_xgb.predict(X_test)))


Ambos presentan metricas muy parecidas pero sin el PCA como suele tener de consecuente al reducir perdemos informacion las metricas sin el pca nos dicen que es mejor el modelo

### Modelo 3.1
##### **Aplicaremos el modelo PCA para la reducción de complejidad de nuestro modelo, con cluesterización** HACER

In [2]:
from sklearn.cluster import KMeans

df = pd.read_csv('./data/processed/Churn_Modelling.csv')
df.drop(columns=['RowNumber', 'CustomerId', 'Surname',], inplace=True)

numeric_features = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']
variables_sin_modificacion = ['HasCrCard', 'IsActiveMember', 'Exited']
categorical_features = ['Geography', 'Gender']

# Definir transformadores para características numéricas y categóricas
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder()

# Crear un ColumnTransformer para aplicar transformaciones a diferentes columnas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Crear el pipeline con el preprocesador
pipeline = make_pipeline(preprocessor)
# Aplicar el preprocesador a los datos
X_processed = pipeline.fit_transform(df)

# Obtener los nombres de las columnas después de aplicar OneHotEncoder
encoded_categorical_columns = preprocessor.named_transformers_['cat']\
    .get_feature_names_out(input_features=categorical_features)

# Combinar los nombres de las columnas numéricas y categóricas
processed_columns = numeric_features + list(encoded_categorical_columns)

# Crear DataFrame con los datos procesados y los nombres de las columnas
processed_df = pd.DataFrame(X_processed, columns=processed_columns)
concatenated_series  = pd.concat((processed_df, df[[ 'HasCrCard', 'IsActiveMember', 'Exited']]), axis=1)

# Inicializar y ajustar el modelo K-Means
n_clusters = 3  # Lo voy a dividir en tres
kmeans = KMeans(n_clusters=n_clusters, random_state=24)
kmeans.fit(concatenated_series)

# Obtener las etiquetas de cluster asignadas a cada punto de datos en el conjunto de entrenamiento
train_labels = kmeans.labels_
# Inicializar un diccionario para almacenar los DataFrames de cada cluster
dict_cluster_dfs = {}

# Iterar sobre cada cluster
for cluster_num in range(n_clusters):
    # Obtener los índices de las filas asignadas al cluster actual
    indices_current_cluster = [i for i, label in enumerate(train_labels) if label == cluster_num]
    # Crear un nuevo DataFrame con las filas del cluster actual
    df_current_cluster = concatenated_series.iloc[indices_current_cluster]
    # Almacenar el DataFrame del cluster actual en el diccionario
    dict_cluster_dfs[cluster_num] = df_current_cluster

# Verificar los DataFrames creados
for cluster_num, df_cluster in dict_cluster_dfs.items():
    print(f"DataFrame para el Cluster {cluster_num}:")
    print(df_cluster.head())  # Puedes imprimir o realizar otras operaciones aquí



DataFrame para el Cluster 0:
    CreditScore       Age    Tenure   Balance  NumOfProducts  EstimatedSalary  \
2     -1.536794  0.293517  1.032908  1.333053       2.527057         0.240687   
5     -0.057205  0.484225  1.032908  0.597329       0.807737         0.863650   
10    -1.267778 -0.755372  0.341352  0.409185       0.807737        -0.346200   
31    -1.216044 -0.278604  0.687130  0.141452      -0.911583         0.984942   
32    -1.009109  0.198164  1.378686  0.538938       0.807737        -0.316331   

    Geography_France  Geography_Germany  Geography_Spain  Gender_Female  \
2                1.0                0.0              0.0            1.0   
5                0.0                0.0              1.0            0.0   
10               1.0                0.0              0.0            0.0   
31               1.0                0.0              0.0            0.0   
32               0.0                1.0              0.0            0.0   

    Gender_Male  HasCrCard  IsAct

In [3]:
X = dict_cluster_dfs[0].drop(columns='Exited')
y = dict_cluster_dfs[0]['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=24)

# Aplicar SMOTE solo al conjunto de entrenamiento
smote = SMOTE(random_state=24)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train , y_train)

BaseLine(X_train_resampled, y_train_resampled, cv=5, metricas_cross_validate= ['accuracy', 'f1_macro', 'recall_macro', 'precision_macro'])

In [103]:
grid_2 = {
    'n_estimators': [100, 150, 200],
    'learning_rate': [0.01, 0.05],
    'max_depth': [4, 6, 8],
    'subsample': [0.7, 0.9],
    'colsample_bytree': [0.7, 0.9]
}


xgbBoost_clust0 = xgb.XGBClassifier()

grd_search_xgbBoost_clust0 = GridSearchCV(xgbBoost_clust0,
                           grid_2,
                           cv=5,
                           scoring='precision_macro',
                           n_jobs=-1
                          )

grd_search_xgbBoost_clust0.fit(X_train_resampled, y_train_resampled)

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.7, 0.9],
                         'learning_rate': [0.01, 0.05], 'max_depth': [4, 6, 8],
                         'n_estimators': [100, 150, 200],
                         'subsample': [0.7, 0.9]},
             scoring='precision_macro')

In [104]:
grd_search_xgbBoost_clust0.best_params_

{'colsample_bytree': 0.9,
 'learning_rate': 0.05,
 'max_depth': 8,
 'n_estimators': 200,
 'subsample': 0.7}

In [105]:

print(classification_report(y_test, grd_search_xgbBoost_clust0.predict(X_test) ))

              precision    recall  f1-score   support

           0       0.93      0.88      0.91       631
           1       0.53      0.66      0.59       124

    accuracy                           0.85       755
   macro avg       0.73      0.77      0.75       755
weighted avg       0.86      0.85      0.85       755



In [106]:
# # Guardar el modelo en un archivo
with open('./modelos/modelo_Xgboost_clust0.pkl', 'wb') as f:
    pickle.dump(grd_search_xgbBoost_clust0, f)
    
# # # Cargar el modelo desde el archivo
# with open('./modelos/modelo_CatBoost_sinfeature_1.pkl', 'rb') as f:
#     loaded_model_xgb = pickle.load(f)

In [4]:
grid_4 = {
    'iterations': [100, 500],
    'learning_rate': [0.01, 0.05],
    'depth': [3, 5, 7],
    'l2_leaf_reg': [1, 4, 7],
    'bagging_temperature': [0, 10, 20]
}

CatBoost_clust0 = CatBoostClassifier()

grd_search_CatBoost_clust0 = GridSearchCV(CatBoost_clust0,
                           grid_4,
                           cv=5,
                           scoring='precision_macro',
                           n_jobs=-1
                          )

grd_search_CatBoost_clust0.fit(X_train_resampled, y_train_resampled)

0:	learn: 0.6696080	total: 161ms	remaining: 1m 20s
1:	learn: 0.6488988	total: 182ms	remaining: 45.4s
2:	learn: 0.6288342	total: 201ms	remaining: 33.2s
3:	learn: 0.6115468	total: 220ms	remaining: 27.2s
4:	learn: 0.5949088	total: 238ms	remaining: 23.6s
5:	learn: 0.5805268	total: 257ms	remaining: 21.1s
6:	learn: 0.5674773	total: 275ms	remaining: 19.4s
7:	learn: 0.5564511	total: 293ms	remaining: 18s
8:	learn: 0.5466598	total: 310ms	remaining: 16.9s
9:	learn: 0.5367531	total: 326ms	remaining: 16s
10:	learn: 0.5275127	total: 346ms	remaining: 15.4s
11:	learn: 0.5194465	total: 367ms	remaining: 14.9s
12:	learn: 0.5120243	total: 390ms	remaining: 14.6s
13:	learn: 0.5058226	total: 420ms	remaining: 14.6s
14:	learn: 0.4935238	total: 450ms	remaining: 14.6s
15:	learn: 0.4882071	total: 470ms	remaining: 14.2s
16:	learn: 0.4835663	total: 496ms	remaining: 14.1s
17:	learn: 0.4769066	total: 516ms	remaining: 13.8s
18:	learn: 0.4719072	total: 536ms	remaining: 13.6s
19:	learn: 0.4647270	total: 555ms	remaining:

GridSearchCV(cv=5,
             estimator=<catboost.core.CatBoostClassifier object at 0x0000021510898510>,
             n_jobs=-1,
             param_grid={'bagging_temperature': [0, 10, 20], 'depth': [3, 5, 7],
                         'iterations': [100, 500], 'l2_leaf_reg': [1, 4, 7],
                         'learning_rate': [0.01, 0.05]},
             scoring='precision_macro')

In [5]:
grd_search_CatBoost_clust0.best_params_

{'bagging_temperature': 0,
 'depth': 7,
 'iterations': 500,
 'l2_leaf_reg': 4,
 'learning_rate': 0.05}

In [6]:
print(classification_report(y_test, grd_search_CatBoost_clust0.predict(X_test) ))

              precision    recall  f1-score   support

           0       0.85      0.89      0.87       476
           1       0.59      0.50      0.54       145

    accuracy                           0.80       621
   macro avg       0.72      0.69      0.71       621
weighted avg       0.79      0.80      0.79       621



In [7]:
# # Guardar el modelo en un archivo
with open('./modelos/modelo_Catboost_clust0.pkl', 'wb') as f:
    pickle.dump(grd_search_CatBoost_clust0, f)
    
# # # Cargar el modelo desde el archivo
# with open('./modelos/modelo_CatBoost_sinfeature_1.pkl', 'rb') as f:
#     loaded_model_xgb = pickle.load(f)

************

In [8]:
X = dict_cluster_dfs[1].drop(columns='Exited')
y = dict_cluster_dfs[1]['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=24)

# Aplicar SMOTE solo al conjunto de entrenamiento
smote = SMOTE(random_state=24)
X_train_resampled_clust1, y_train_resampled_clust1 = smote.fit_resample(X_train , y_train)

# BaseLine(X_train_resampled_clust1, y_train_resampled_clust1, cv=5, metricas_cross_validate= ['accuracy', 'f1_macro', 'recall_macro', 'precision_macro'])

In [9]:
grid_5 = {
    'n_estimators': [50, 100, 200],
    'max_depth': [20, 30, 40,  None],
    'min_samples_split': [2, 10],
    'min_samples_leaf': [1, 5, 10],
    'max_features': ['auto', 'sqrt'],
    'bootstrap': [True, False]
}

forest_clus1 = RandomForestClassifier()

grd_search_forest_clus1 = GridSearchCV(forest_clus1,
                           grid_5,
                           cv=5,
                           scoring='precision_macro',
                           n_jobs=-1
                          )

grd_search_forest_clus1.fit(X_train_resampled_clust1, y_train_resampled_clust1)

GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'max_depth': [20, 30, 40, None],
                         'max_features': ['auto', 'sqrt'],
                         'min_samples_leaf': [1, 5, 10],
                         'min_samples_split': [2, 10],
                         'n_estimators': [50, 100, 200]},
             scoring='precision_macro')

In [10]:
grd_search_forest_clus1.best_params_

{'bootstrap': False,
 'max_depth': None,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 100}

In [11]:
print(classification_report(y_test, grd_search_forest_clus1.predict(X_test) ))

              precision    recall  f1-score   support

           0       0.87      0.90      0.88       490
           1       0.58      0.50      0.54       136

    accuracy                           0.81       626
   macro avg       0.72      0.70      0.71       626
weighted avg       0.80      0.81      0.81       626



In [12]:
# # Guardar el modelo en un archivo
with open('./modelos/modelo_forest_clust1.pkl', 'wb') as f:
    pickle.dump(grd_search_forest_clus1, f)
    
# # # Cargar el modelo desde el archivo
# with open('./modelos/modelo_CatBoost_sinfeature_1.pkl', 'rb') as f:
#     loaded_model_xgb = pickle.load(f)

In [13]:
grid_2 = {
    'n_estimators': [100, 150, 200],
    'learning_rate': [0.01, 0.05],
    'max_depth': [4, 6, 8],
    'subsample': [0.7, 0.9],
    'colsample_bytree': [0.7, 0.9]
}

xgbBoost_clust1 = xgb.XGBClassifier()

grd_search_xgbBoost_clust1 = GridSearchCV(xgbBoost_clust1,
                           grid_2,
                           cv=5,
                           scoring='precision_macro',
                           n_jobs=-1
                          )

grd_search_xgbBoost_clust1.fit(X_train_resampled_clust1, y_train_resampled_clust1)

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.7, 0.9],
                         'learning_rate': [0.01, 0.05], 'max_depth': [4, 6, 8],
                         'n_estimators': [100, 150, 200],
                         'subsample': [0.7, 0.9]},
             scoring='precision_macro')

In [14]:
grd_search_xgbBoost_clust1.best_params_

{'colsample_bytree': 0.7,
 'learning_rate': 0.05,
 'max_depth': 8,
 'n_estimators': 200,
 'subsample': 0.9}

In [15]:
print(classification_report(y_test, grd_search_xgbBoost_clust1.predict(X_test) ))

              precision    recall  f1-score   support

           0       0.87      0.88      0.87       490
           1       0.55      0.52      0.53       136

    accuracy                           0.80       626
   macro avg       0.71      0.70      0.70       626
weighted avg       0.80      0.80      0.80       626



In [16]:
# # Guardar el modelo en un archivo
with open('./modelos/modelo_xgboost_clust1.pkl', 'wb') as f:
    pickle.dump(grd_search_xgbBoost_clust1, f)
    
# # # Cargar el modelo desde el archivo
# with open('./modelos/modelo_CatBoost_sinfeature_1.pkl', 'rb') as f:
#     loaded_model_xgb = pickle.load(f)

****************

In [17]:
X = dict_cluster_dfs[2].drop(columns='Exited')
y = dict_cluster_dfs[2]['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=24)

# Aplicar SMOTE solo al conjunto de entrenamiento
smote = SMOTE(random_state=24)
X_train_resampled_clust2, y_train_resampled_clust2 = smote.fit_resample(X_train , y_train)

# BaseLine(X_train_resampled_clust2, y_train_resampled_clust2, cv=5, metricas_cross_validate= ['accuracy', 'f1_macro', 'recall_macro', 'precision_macro'])

In [18]:
grid_6 = {
    'n_estimators': [100, 150, 200, 300],
    'learning_rate': [0.01, 0.05],
    'max_depth': [4, 6, 8],
    'subsample': [0.7, 0.9],
    'colsample_bytree': [0.7, 0.9]
}

xgbBoost_clust2 = xgb.XGBClassifier()

grd_search_xgbBoost_clust2 = GridSearchCV(xgbBoost_clust2,
                           grid_6,
                           cv=5,
                           scoring='precision_macro',
                           n_jobs=-1
                          )

grd_search_xgbBoost_clust2.fit(X_train_resampled_clust2, y_train_resampled_clust2)

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.7, 0.9],
                         'learning_rate': [0.01, 0.05], 'max_depth': [4, 6, 8],
                         'n_estimators': [100, 150, 200, 300],
                         'subsample': [0.7, 0.9]},
             scoring='precision_macro')

In [19]:
grd_search_xgbBoost_clust2.best_params_

{'colsample_bytree': 0.7,
 'learning_rate': 0.05,
 'max_depth': 8,
 'n_estimators': 300,
 'subsample': 0.7}

In [20]:
print(classification_report(y_test, grd_search_xgbBoost_clust2.predict(X_test) ))

              precision    recall  f1-score   support

           0       0.93      0.95      0.94       631
           1       0.70      0.64      0.67       124

    accuracy                           0.90       755
   macro avg       0.81      0.79      0.80       755
weighted avg       0.89      0.90      0.89       755



In [21]:
# # Guardar el modelo en un archivo
with open('./modelos/modelo_xgboost_clust2.pkl', 'wb') as f:
    pickle.dump(grd_search_xgbBoost_clust2, f)
    
# # # Cargar el modelo desde el archivo
# with open('./modelos/modelo_CatBoost_sinfeature_1.pkl', 'rb') as f:
#     loaded_model_xgb = pickle.load(f)

In [22]:
grid_7 = {
    'iterations': [100, 500],
    'learning_rate': [0.01, 0.05],
    'depth': [3, 5, 7],
    'l2_leaf_reg': [1, 4, 7],
    'bagging_temperature': [0, 10, 20]
}

CatBoost_clust2 = CatBoostClassifier()

grd_search_CatBoost_clust2 = GridSearchCV(CatBoost_clust2,
                           grid_4,
                           cv=5,
                           scoring='precision_macro',
                           n_jobs=-1
                          )

grd_search_CatBoost_clust2.fit(X_train_resampled_clust2, y_train_resampled_clust2)

0:	learn: 0.6417503	total: 81.6ms	remaining: 40.7s
1:	learn: 0.5974326	total: 119ms	remaining: 29.8s
2:	learn: 0.5636644	total: 180ms	remaining: 29.8s
3:	learn: 0.5284323	total: 225ms	remaining: 27.9s
4:	learn: 0.4989539	total: 274ms	remaining: 27.1s
5:	learn: 0.4738438	total: 305ms	remaining: 25.1s
6:	learn: 0.4512546	total: 342ms	remaining: 24.1s
7:	learn: 0.4337764	total: 375ms	remaining: 23.1s
8:	learn: 0.4185816	total: 410ms	remaining: 22.4s
9:	learn: 0.4034511	total: 547ms	remaining: 26.8s
10:	learn: 0.3925974	total: 601ms	remaining: 26.7s
11:	learn: 0.3809298	total: 647ms	remaining: 26.3s
12:	learn: 0.3696879	total: 713ms	remaining: 26.7s
13:	learn: 0.3610525	total: 763ms	remaining: 26.5s
14:	learn: 0.3533157	total: 811ms	remaining: 26.2s
15:	learn: 0.3459399	total: 880ms	remaining: 26.6s
16:	learn: 0.3403551	total: 934ms	remaining: 26.5s
17:	learn: 0.3341920	total: 976ms	remaining: 26.1s
18:	learn: 0.3280144	total: 1.02s	remaining: 25.7s
19:	learn: 0.3237345	total: 1.06s	remain

GridSearchCV(cv=5,
             estimator=<catboost.core.CatBoostClassifier object at 0x0000021523ACD710>,
             n_jobs=-1,
             param_grid={'bagging_temperature': [0, 10, 20], 'depth': [3, 5, 7],
                         'iterations': [100, 500], 'l2_leaf_reg': [1, 4, 7],
                         'learning_rate': [0.01, 0.05]},
             scoring='precision_macro')

In [23]:
grd_search_CatBoost_clust2.best_params_

{'bagging_temperature': 0,
 'depth': 7,
 'iterations': 500,
 'l2_leaf_reg': 1,
 'learning_rate': 0.05}

In [24]:
print(classification_report(y_test, grd_search_CatBoost_clust2.predict(X_test) ))

              precision    recall  f1-score   support

           0       0.92      0.94      0.93       631
           1       0.66      0.59      0.62       124

    accuracy                           0.88       755
   macro avg       0.79      0.76      0.78       755
weighted avg       0.88      0.88      0.88       755



In [25]:
# # Guardar el modelo en un archivo
with open('./modelos/modelo_xgboost_clust2.pkl', 'wb') as f:
    pickle.dump(grd_search_CatBoost_clust2, f)
    
# # # Cargar el modelo desde el archivo
# with open('./modelos/modelo_CatBoost_sinfeature_1.pkl', 'rb') as f:
#     loaded_model_xgb = pickle.load(f)

### Modelo 4
##### **Aplicaremos el modelo PCA para la reducción de complejidad de nuestro modelo. Sin features y con cluesterización manual por Geography**

In [27]:
df = pd.read_csv('./data/processed/Churn_Modelling.csv')

df['is_male'] = df['Gender'].replace({'Female': 0, 'Male' : 1})
df.drop(columns=['RowNumber', 'CustomerId', 'Surname', 'Gender'], inplace=True)

df_france = df[df['Geography'] == 'France']
df_spain = df[df['Geography'] == 'Spain']
df_germany = df[df['Geography'] == 'Germany']


In [28]:
X = df_france.drop(columns='Exited')
y = df_france['Exited']

numeric_features = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']
variables_sin_modificacion =  df_france[[ 'HasCrCard', 'IsActiveMember', 'is_male']]

# Definir transformadores para características numéricas y categóricas
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder()

# Crear un ColumnTransformer para aplicar transformaciones a diferentes columnas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)
    ]
)
# Crear el pipeline con el preprocesador
pipeline = make_pipeline(preprocessor)
# Aplicar el preprocesador a los datos
X_processed = pipeline.fit_transform(X)

# Crear DataFrame con los datos procesados y los nombres de las columnas
processed_df = pd.DataFrame(X_processed, columns=numeric_features)
concatenated_series_france  = pd.concat((processed_df, variables_sin_modificacion.reset_index(drop=True) ), axis=1) #


# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(concatenated_series_france, y, test_size=0.2, random_state=24)

pca = PCA(n_components=7)
X_train_pca  = pca.fit_transform(X_train)

# Aplicar SMOTE solo al conjunto de entrenamiento
smote = SMOTE(random_state=24)
X_train_resampled_france, y_train_resampled_france = smote.fit_resample(X_train_pca , y_train)

BaseLine(X_train_resampled_france, y_train_resampled_france, cv=5, metricas_cross_validate= ['accuracy', 'f1_macro', 'recall_macro', 'precision_macro'])


Learning rate set to 0.021129
0:	learn: 0.6823682	total: 80.5ms	remaining: 1m 20s
1:	learn: 0.6732128	total: 130ms	remaining: 1m 5s
2:	learn: 0.6651122	total: 218ms	remaining: 1m 12s
3:	learn: 0.6577872	total: 315ms	remaining: 1m 18s
4:	learn: 0.6488133	total: 366ms	remaining: 1m 12s
5:	learn: 0.6406462	total: 419ms	remaining: 1m 9s
6:	learn: 0.6326557	total: 464ms	remaining: 1m 5s
7:	learn: 0.6248873	total: 563ms	remaining: 1m 9s
8:	learn: 0.6165658	total: 605ms	remaining: 1m 6s
9:	learn: 0.6102739	total: 640ms	remaining: 1m 3s
10:	learn: 0.6036043	total: 675ms	remaining: 1m
11:	learn: 0.5963584	total: 735ms	remaining: 1m
12:	learn: 0.5905374	total: 784ms	remaining: 59.5s
13:	learn: 0.5848419	total: 829ms	remaining: 58.4s
14:	learn: 0.5798780	total: 890ms	remaining: 58.4s
15:	learn: 0.5745589	total: 935ms	remaining: 57.5s
16:	learn: 0.5687214	total: 987ms	remaining: 57.1s
17:	learn: 0.5631062	total: 1.02s	remaining: 55.9s
18:	learn: 0.5591635	total: 1.08s	remaining: 55.7s
19:	learn: 0

,Modelo,Score
34,XGBoost_recall,0.895731
35,XGBoost_precision,0.897454
33,XGBoost_f1,0.895615
32,XGBoost_accuracy,0.895722
42,SVC_recall,0.813002
43,SVC_precision,0.813389
41,SVC_f1,0.812944
40,SVC_accuracy,0.813004
6,Random Forest_recall,0.898254
7,Random Forest_precision,0.899453


XGBoost y Random Forest

Hacer un pequeño resumen y análisis

In [29]:
grid_3 = {
    'n_estimators': [100, 200],
    'max_depth': [20, 30],
    'min_samples_split': [2, 10],
    'min_samples_leaf': [1, 5],
    'max_features': ['auto', 'sqrt'],
    'bootstrap': [False]
}

forest_france = RandomForestClassifier()

grd_search_france_forest = GridSearchCV(forest_france,
                           grid_3,
                           cv=5,
                           scoring='precision_macro',
                           n_jobs=-1
                          )

grd_search_france_forest.fit(X_train_resampled_france, y_train_resampled_france)

GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'bootstrap': [False], 'max_depth': [20, 30],
                         'max_features': ['auto', 'sqrt'],
                         'min_samples_leaf': [1, 5],
                         'min_samples_split': [2, 10],
                         'n_estimators': [100, 200]},
             scoring='precision_macro')

In [34]:
X_train_resampled_france.shape

(6722, 7)

In [38]:
print('Mejores hiperparámetros obtenidos:', grd_search_france_forest.best_params_)
X_test_pca  = pca.fit_transform(X_test)
print(classification_report(y_test, grd_search_france_forest.predict(X_test_pca) ))

# # # Guardar el modelo en un archivo
# with open('./modelos/modelo_forest_france.pkl', 'wb') as f:
#     pickle.dump(grd_search_france_forest, f)
    
# # # Cargar el modelo desde el archivo
# with open('./modelos/modelo_CatBoost_sinfeature_1.pkl', 'rb') as f:
#     loaded_model_xgb = pickle.load(f)

Mejores hiperparámetros obtenidos: {'bootstrap': False, 'max_depth': 20, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
              precision    recall  f1-score   support

           0       0.89      0.79      0.84       843
           1       0.30      0.47      0.36       160

    accuracy                           0.74      1003
   macro avg       0.59      0.63      0.60      1003
weighted avg       0.79      0.74      0.76      1003



In [39]:
grid_2 = {
    'n_estimators': [150, 200],
    'learning_rate': [0.01, 0.05],
    'max_depth': [6, 8],
    'subsample': [0.7, 0.9],
    'colsample_bytree': [0.7, 0.9]
}

xgb_france = xgb.XGBClassifier()

grid_search_france_xgb = GridSearchCV(xgb_france,
                           grid_2,
                           cv=5,
                           scoring='precision_macro',
                           n_jobs=-1
                          )

grid_search_france_xgb.fit(X_train_resampled_france, y_train_resampled_france)

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.7, 0.9],
                         'learning_rate': [0.01, 0.05], 'max_depth': [6, 8],
                         'n_estimators': [150, 200], 'subsample': [0.7, 0.9]},
             scoring='precision_macro')

In [40]:
print('Mejores hiperparámetros obtenidos:', grid_search_france_xgb.best_params_)

print(classification_report(y_test, grid_search_france_xgb.predict(X_test_pca) ))

# # Guardar el modelo en un archivo
with open('./modelos/modelo_Xgboost_france.pkl', 'wb') as f:
    pickle.dump(grid_search_france_xgb, f)
    
# # # Cargar el modelo desde el archivo
# with open('./modelos/modelo_CatBoost_sinfeature_1.pkl', 'rb') as f:
#     loaded_model_xgb = pickle.load(f)

Mejores hiperparámetros obtenidos: {'colsample_bytree': 0.9, 'learning_rate': 0.05, 'max_depth': 8, 'n_estimators': 200, 'subsample': 0.9}
              precision    recall  f1-score   support

           0       0.89      0.52      0.65       843
           1       0.21      0.66      0.31       160

    accuracy                           0.54      1003
   macro avg       0.55      0.59      0.48      1003
weighted avg       0.78      0.54      0.60      1003



Elegir el mejor modelo para Francia y predecir 

**********

In [41]:
X = df_spain.drop(columns='Exited')
y = df_spain['Exited']

numeric_features = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']
variables_sin_modificacion =  df_spain[[ 'HasCrCard', 'IsActiveMember', 'is_male']]

# Definir transformadores para características numéricas y categóricas
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder()

# Crear un ColumnTransformer para aplicar transformaciones a diferentes columnas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)
    ]
)
# Crear el pipeline con el preprocesador
pipeline = make_pipeline(preprocessor)
# Aplicar el preprocesador a los datos
X_processed = pipeline.fit_transform(X)

# Crear DataFrame con los datos procesados y los nombres de las columnas
processed_df = pd.DataFrame(X_processed, columns=numeric_features)
concatenated_series_spain  = pd.concat([processed_df, variables_sin_modificacion.reset_index(drop=True) ], axis=1) #

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(concatenated_series_spain, y, test_size=0.2, random_state=24)

pca = PCA(n_components=7)
X_train_pca = pca.fit_transform(X_train)
X_test_pca  = pca.transform(X_test)

# Aplicar SMOTE solo al conjunto de entrenamiento
smote = SMOTE(random_state=24)
X_train_resampled_spain, y_train_resampled_spain = smote.fit_resample(X_train_pca, y_train)

# BaseLine(X_train_resampled_spain, y_train_resampled_spain, cv=5, metricas_cross_validate= ['accuracy', 'f1_macro', 'recall_macro', 'precision_macro'])

In [42]:
grid_3 = {
    'n_estimators': [50, 100],
    'max_depth': [30, 40],
    'min_samples_split': [2, 10],
    'min_samples_leaf': [1, 5],
    'max_features': ['auto', 'sqrt'],
    'bootstrap': [True, False]
}

forest_spain = RandomForestClassifier()

grid_search_spain_forest = GridSearchCV(forest_spain,
                           grid_3,
                           cv=5,
                           scoring='precision_macro',
                           n_jobs=-1
                          )

grid_search_spain_forest.fit(X_train_resampled_spain, y_train_resampled_spain)

GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'bootstrap': [True, False], 'max_depth': [30, 40],
                         'max_features': ['auto', 'sqrt'],
                         'min_samples_leaf': [1, 5],
                         'min_samples_split': [2, 10],
                         'n_estimators': [50, 100]},
             scoring='precision_macro')

In [43]:
print('Los mejores hiperparámetros obtenidos:' , grid_search_spain_forest.best_params_)
print(classification_report(y_test, grid_search_spain_forest.predict(X_test_pca) ))
# # Guardar el modelo en un archivo
with open('./modelos/modelo_forest_spain.pkl', 'wb') as f:
    pickle.dump(grid_search_spain_forest, f)
    
# # # Cargar el modelo desde el archivo
# with open('./modelos/modelo_CatBoost_sinfeature_1.pkl', 'rb') as f:
#     loaded_model_xgb = pickle.load(f)

Los mejores hiperparámetros obtenidos: {'bootstrap': False, 'max_depth': 30, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
              precision    recall  f1-score   support

           0       0.92      0.91      0.91       417
           1       0.54      0.56      0.55        79

    accuracy                           0.85       496
   macro avg       0.73      0.73      0.73       496
weighted avg       0.86      0.85      0.86       496



In [44]:
grid_2 = {
    'n_estimators': [100, 150, 200],
    'learning_rate': [0.01, 0.05],
    'max_depth': [4, 6, 8],
    'subsample': [0.7, 0.9],
    'colsample_bytree': [0.7, 0.9]
}

xgb_spain = xgb.XGBClassifier()

grid_search_spain_xgb = GridSearchCV(xgb_spain,
                           grid_2,
                           cv=5,
                           scoring='precision_macro',
                           n_jobs=-1
                          )

grid_search_spain_xgb.fit(X_train_resampled_spain, y_train_resampled_spain)

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.7, 0.9],
                         'learning_rate': [0.01, 0.05], 'max_depth': [4, 6, 8],
                         'n_estimators': [100, 150, 200],
                         'subsample': [0.7, 0.9]},
             scoring='precision_macro')

In [46]:
print('Los mejores hiperparámetros obtenidos:' ,grid_search_spain_xgb.best_params_)

print(classification_report(y_test, grid_search_spain_xgb.predict(X_test_pca) ))
# # Guardar el modelo en un archivo
with open('./modelos/modelo_Xgboost_spain.pkl', 'wb') as f:
    pickle.dump(grid_search_spain_xgb, f)
    
# # # Cargar el modelo desde el archivo
# with open('./modelos/modelo_CatBoost_sinfeature_1.pkl', 'rb') as f:
#     loaded_model_xgb = pickle.load(f)

Los mejores hiperparámetros obtenidos: {'colsample_bytree': 0.9, 'learning_rate': 0.05, 'max_depth': 8, 'n_estimators': 200, 'subsample': 0.9}
              precision    recall  f1-score   support

           0       0.92      0.89      0.90       417
           1       0.51      0.61      0.55        79

    accuracy                           0.84       496
   macro avg       0.71      0.75      0.73       496
weighted avg       0.86      0.84      0.85       496



Hacer un pequeño analisis y resumen de lo hecho 
- Elegir el mejor modelo para España y predecir

*******************

In [48]:
X = df_germany.drop(columns='Exited')
y = df_germany['Exited']

numeric_features = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']
variables_sin_modificacion =  df_germany[[ 'HasCrCard', 'IsActiveMember', 'is_male']]

# Definir transformadores para características numéricas y categóricas
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder()

# Crear un ColumnTransformer para aplicar transformaciones a diferentes columnas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)
    ]
)
# Crear el pipeline con el preprocesador
pipeline = make_pipeline(preprocessor)
# Aplicar el preprocesador a los datos
X_processed = pipeline.fit_transform(X)

# Crear DataFrame con los datos procesados y los nombres de las columnas
processed_df = pd.DataFrame(X_processed, columns=numeric_features)
concatenated_series_germany  = pd.concat([processed_df, variables_sin_modificacion.reset_index(drop=True) ], axis=1) #

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(concatenated_series_germany, y, test_size=0.2, random_state=24)

pca = PCA(n_components=7)
X_train_pca = pca.fit_transform(X_train)
X_test_pca  = pca.transform(X_test)

# Aplicar SMOTE solo al conjunto de entrenamiento
smote = SMOTE(random_state=24)
X_train_resampled_germany, y_train_resampled_germany = smote.fit_resample(X_train_pca, y_train)

# BaseLine(X_train_resampled_germany, y_train_resampled_germany, cv=5, metricas_cross_validate= ['accuracy', 'f1_macro', 'recall_macro', 'precision_macro'])

In [49]:
grid_2 = {
    'n_estimators': [100, 150],
    'max_depth': [10, 15],
    'min_samples_split': [8, 10],
    'min_samples_leaf': [3, 4],
    'bootstrap': [True]
}

forest_german = RandomForestClassifier()

grid_search_german_forest = GridSearchCV(forest_german,
                           grid_2,
                           cv=5,
                           scoring='precision_macro',
                           n_jobs=-1
                          )

grid_search_german_forest.fit(X_train_resampled_germany, y_train_resampled_germany)

GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'bootstrap': [True], 'max_depth': [10, 15],
                         'min_samples_leaf': [3, 4],
                         'min_samples_split': [8, 10],
                         'n_estimators': [100, 150]},
             scoring='precision_macro')

In [53]:
print('Los mejores hiperparámetros obtenidos:' , grid_search_german_forest.best_params_)

print(classification_report(y_test, grid_search_german_forest.predict(X_test_pca) ))
# # Guardar el modelo en un archivo
with open('./modelos/modelo_forest_german.pkl', 'wb') as f:
    pickle.dump(grid_search_german_forest, f)
    
# # # Cargar el modelo desde el archivo
# with open('./modelos/modelo_CatBoost_sinfeature_1.pkl', 'rb') as f:
#     loaded_model_xgb = pickle.load(f)

Los mejores hiperparámetros obtenidos: {'bootstrap': True, 'max_depth': 15, 'min_samples_leaf': 3, 'min_samples_split': 10, 'n_estimators': 100}
              precision    recall  f1-score   support

           0       0.84      0.81      0.82       345
           1       0.61      0.66      0.63       157

    accuracy                           0.76       502
   macro avg       0.73      0.73      0.73       502
weighted avg       0.77      0.76      0.77       502



In [51]:
grid_2 = {
    'n_estimators': [100, 200],
    'learning_rate': [0.01, 0.05],
    'max_depth': [6, 8],
    'subsample': [0.7, 0.9],
    'colsample_bytree': [0.7, 0.9]
}

xgb_german = xgb.XGBClassifier()

grid_search_german_xgb = GridSearchCV(xgb_german,
                           grid_2,
                           cv=5,
                           scoring='precision_macro',
                           n_jobs=-1
                          )

grid_search_german_xgb.fit(X_train_resampled_germany, y_train_resampled_germany)

GridSearchCV(cv=5,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.7, 0.9],
                         'learning_rate': [0.01, 0.05], 'max_depth': [6, 8],
                         'n_estimators': [100, 200], 'subsample': [0.7, 0.9]},
             scoring='precision_macro')

In [52]:
print('Los mejores hiperparámetros obtenidos:' ,grid_search_german_xgb.best_params_)

print(classification_report(y_test, grid_search_german_xgb.predict(X_test_pca) ))
# # Guardar el modelo en un archivo
with open('./modelos/modelo_Xgboost_german.pkl', 'wb') as f:
    pickle.dump(grid_search_german_xgb, f)
    
# # # Cargar el modelo desde el archivo
# with open('./modelos/modelo_CatBoost_sinfeature_1.pkl', 'rb') as f:
#     loaded_model_xgb = pickle.load(f)

Los mejores hiperparámetros obtenidos: {'colsample_bytree': 0.9, 'learning_rate': 0.05, 'max_depth': 8, 'n_estimators': 200, 'subsample': 0.7}
              precision    recall  f1-score   support

           0       0.84      0.81      0.82       345
           1       0.61      0.66      0.63       157

    accuracy                           0.76       502
   macro avg       0.72      0.73      0.73       502
weighted avg       0.76      0.76      0.76       502



Luego de ver las mejores prediccione evitando el sobreajusta entre la cluesterizacion manual y la automatica (KMeans) se elegirá el mejor modelo aplicado por los gridSearch para pasar al archivo train.py

#### Modelo 5